##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Neural machine translation with attention

This notebook trains a sequence to sequence (seq2seq) model for Spanish to English translation. This is an advanced example that assumes some knowledge of sequence to sequence models.

After training the model in this notebook, you will be able to input a Spanish sentence, such as *"¿todavia estan en casa?"*, and return the English translation: *"are you still at home?"*

The translation quality is reasonable for a toy example, but the generated attention plot is perhaps more interesting. This shows which parts of the input sentence has the model's attention while translating:

In [1]:
!pip install tensorflow_addons

     |████████████████████████████████| 706kB 7.5MB/s 


In [2]:
import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import unicodedata
import re
import numpy as np
import os
import io
import time
import ast
import pandas as pd

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


## Download and prepare the dataset

After downloading the dataset, here are the steps we'll take to prepare the data:

1. Add a *start* and *end* token to each sentence.
2. Clean the sentences by removing special characters.
3. Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
4. Pad each sentence to a maximum length.

In [4]:

import os
#os.system("git clone https://github.com/soerenetler/squad-question-generation.git")
#os.system("git clone https://github.com/soerenetler/spacy_target_answer.git")
#os.system("git clone https://github.com/soerenetler/crf_target_answer.git")

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
path_to_folder = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/squad/question_answer/"
path_to_model = "/content/gdrive/MyDrive/mt-qg-data/00_models/qg_attention/squad/"

In [6]:
class QGDataset:
  def __init__(self, problem_type='squad', data_folder="/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/"):
    self.problem_type = problem_type
    self.dev_path = data_folder + problem_type + "/question_answer/dev.csv"
    self.train_path = data_folder + problem_type + "/question_answer/train.csv"
    self.inp_lang_tokenizer = None
    self.targ_lang_tokenizer = None

  # Converts the unicode file to ascii
  def unicode_to_ascii(self, s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')
  
  def preprocess_sentence(self, w):
    w_result = ['<start>']
    for t in w:
      t = self.unicode_to_ascii(t.lower().strip())

      # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
      #t = re.sub(r"[^a-zA-Z\d?.!,¿]+", " ", t)

      #t = t.strip()
      if t != '':
        w_result.append(t)
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w_result.append('<end>')
    return w_result

  # 1. Remove the accents
  # 2. Clean the sentences
  # 3. Return word pairs in the format: [ENGLISH, SPANISH]
  def create_dataset(self, path):
    df = pd.read_csv(path)
    df["answer_sentence_token"] = [ast.literal_eval(t) for t in df["answer_sentence_token"]]
    df["question_token"] = [ast.literal_eval(t) for t in df["question_token"]]

    #from plotly import express
    #display(express.histogram(x=[len(t) for t in df["answer_sentence_token"]]))

    #display(df["answer_sentence_token"].head(50))

    sentence_pairs = zip(df["answer_sentence_token"].apply(self.preprocess_sentence), df["question_token"].apply(self.preprocess_sentence))
    

    return zip(*sentence_pairs)

  def tokenize(self, lang, maxlen, max_vocab):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        #filters='',
        num_words=max_vocab,
        oov_token="<oov>")
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                          maxlen=maxlen,
                                                          padding='post')

    return tensor, lang_tokenizer

  def load_dataset(self):
    # creating cleaned input, output pairs
    inp_lang_train, targ_lang_train = self.create_dataset(self.train_path)
    inp_lang_dev, targ_lang_dev = self.create_dataset(self.dev_path)

    print(inp_lang_train[-1])
    print(targ_lang_train[-1])

    input_tensor, inp_lang_tokenizer = self.tokenize(inp_lang_train, max_length_inp, max_vocab_inp)
    target_tensor, targ_lang_tokenizer = self.tokenize(targ_lang_train, max_length_targ, max_vocab_targ)

    print(inp_lang_dev[0])
    print(targ_lang_dev[0])
    input_tensor_dev = inp_lang_tokenizer.texts_to_sequences(inp_lang_dev)
    target_tensor_dev = targ_lang_tokenizer.texts_to_sequences(targ_lang_dev)
    print(input_tensor_dev[0])
    print(target_tensor_dev[0])

    return input_tensor, target_tensor, input_tensor_dev, target_tensor_dev, inp_lang_tokenizer, targ_lang_tokenizer

In [7]:
answer_sentence = ['3245', 'two', 'months', 'later', 'the', 'band', 'got', 'signed', 'to', 'a', 'three', 'album', 'deal', 'with', 'spinefarm', ',', 'which', 'left', 'marko', 'displeased', '.']
question_sentence = ['what', 'label', 'were', 'they', 'with', '?']

qg_dataset = QGDataset()
print(qg_dataset.preprocess_sentence(answer_sentence))
print(qg_dataset.preprocess_sentence(question_sentence))

['<start>', '3245', 'two', 'months', 'later', 'the', 'band', 'got', 'signed', 'to', 'a', 'three', 'album', 'deal', 'with', 'spinefarm', ',', 'which', 'left', 'marko', 'displeased', '.', '<end>']
['<start>', 'what', 'label', 'were', 'they', 'with', '?', '<end>']


### Limit the size of the dataset to experiment faster (optional)

Training on the complete dataset of >100,000 sentences will take a long time. To train faster, we can limit the size of the dataset to 30,000 sentences (of course, translation quality degrades with less data):

In [8]:
# max_length of the target tensors
max_length_targ = 20
max_length_inp = 80
max_vocab_inp = 45000
max_vocab_targ = 28000

# Try experimenting with the size of that dataset
input_tensor_train, target_tensor_train, input_tensor_dev, target_tensor_dev, inp_tokenizer, targ_tokenizer = qg_dataset.load_dataset()

['<start>', 'according', 'to', 'the', 'government', 'of', 'botswana', ',', 'trophy', 'hunting', 'is', 'at', 'least', 'partly', 'to', 'blame', 'for', 'this', ',', 'but', 'many', 'other', 'factors', ',', 'such', 'as', 'poaching', ',', 'drought', 'and', 'habitat', 'loss', 'are', 'also', 'to', 'blame', '.', '<end>']
['<start>', 'what', 'else', 'is', 'partly', 'to', 'blame', 'for', 'the', 'declining', 'number', 'of', 'animals', 'in', 'botswana', 'and', 'uganda', '?', '<end>']
['<start>', 'the', 'legend', 'of', 'zelda', ':', 'twilight', 'princess', '(', 'japanese', ':', 'セルタの伝説', 'トワイライトフリンセス', ',', 'hepburn', ':', 'zeruda', 'no', 'densetsu', ':', 'towairaito', 'purinsesu', '?', ')', 'is', 'an', 'action', '-', 'adventure', 'game', 'developed', 'and', 'published', 'by', 'nintendo', 'for', 'the', 'gamecube', 'and', 'wii', 'home', 'video', 'game', 'consoles', '.', '<end>']
['<start>', 'what', 'category', 'of', 'game', 'is', 'legend', 'of', 'zelda', ':', 'twilight', 'princess', '?', '<end>']
[5,

In [9]:
# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_dev), len(target_tensor_dev))

68174 68174 18647 18647


In [10]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [11]:
print ("Input Language; index to word mapping")
convert(inp_tokenizer, input_tensor_dev[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_tokenizer, target_tensor_dev[0])

Input Language; index to word mapping
5 ----> <start>
2 ----> the
5762 ----> legend
4 ----> of
1 ----> <oov>
53 ----> :
44108 ----> twilight
4330 ----> princess
12 ----> (
405 ----> japanese
53 ----> :
1 ----> <oov>
1 ----> <oov>
3 ----> ,
31057 ----> hepburn
53 ----> :
1 ----> <oov>
127 ----> no
1 ----> <oov>
53 ----> :
1 ----> <oov>
1 ----> <oov>
1855 ----> ?
13 ----> )
17 ----> is
30 ----> an
600 ----> action
15 ----> -
7121 ----> adventure
521 ----> game
234 ----> developed
8 ----> and
392 ----> published
19 ----> by
1594 ----> nintendo
20 ----> for
2 ----> the
34010 ----> gamecube
8 ----> and
8877 ----> wii
204 ----> home
597 ----> video
521 ----> game
6115 ----> consoles
7 ----> .
6 ----> <end>

Target Language; index to word mapping
2 ----> <start>
6 ----> what
1749 ----> category
7 ----> of
309 ----> game
10 ----> is
4715 ----> legend
7 ----> of
16507 ----> zelda
1673 ----> :
1 ----> <oov>
3877 ----> princess
4 ----> ?
3 ----> <end>


In [12]:
#!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
#!unzip -q glove.840B.300d.zip

In [13]:
path_to_glove_file = os.path.join("/content/gdrive/MyDrive/mt-qg-data/glove.840B.300d.txt")


embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  


Found 2195884 word vectors.


In [14]:
num_tokens = len(inp_tokenizer.index_word) + 1
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for i, word in inp_tokenizer.index_word.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None and len(embedding_vector) == 300:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 48836 words (14374 misses)


### Create a tf.data dataset

In [15]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 600
vocab_inp_size = len(inp_tokenizer.word_index)+1 # PADDING
vocab_tar_size = len(targ_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [16]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 80]), TensorShape([64, 20]))

## Write the encoder and decoder model

Implement an encoder-decoder model with attention which you can read about in the TensorFlow [Neural Machine Translation (seq2seq) tutorial](https://github.com/tensorflow/nmt). This example uses a more recent set of APIs. This notebook implements the [attention equations](https://github.com/tensorflow/nmt#background-on-the-attention-mechanism) from the seq2seq tutorial. The following diagram shows that each input words is assigned a weight by the attention mechanism which is then used by the decoder to predict the next word in the sentence. The below picture and formulas are an example of attention mechanism from [Luong's paper](https://arxiv.org/abs/1508.04025v5). 

<img src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg" width="500" alt="attention mechanism">

The input is put through an encoder model which gives us the encoder output of shape *(batch_size, max_length, hidden_size)* and the encoder hidden state of shape *(batch_size, hidden_size)*.

Here are the equations that are implemented:

<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg" alt="attention equation 0" width="800">
<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg" alt="attention equation 1" width="800">

This tutorial uses [Bahdanau attention](https://arxiv.org/pdf/1409.0473.pdf) for the encoder. Let's decide on notation before writing the simplified form:

* FC = Fully connected (dense) layer
* EO = Encoder output
* H = hidden state
* X = input to the decoder

And the pseudo-code:

* `score = FC(tanh(FC(EO) + FC(H)))`
* `attention weights = softmax(score, axis = 1)`. Softmax by default is applied on the last axis but here we want to apply it on the *1st axis*, since the shape of score is *(batch_size, max_length, hidden_size)*. `Max_length` is the length of our input. Since we are trying to assign a weight to each input, softmax should be applied on that axis.
* `context vector = sum(attention weights * EO, axis = 1)`. Same reason as above for choosing axis as 1.
* `embedding output` = The input to the decoder X is passed through an embedding layer.
* `merged vector = concat(embedding output, context vector)`
* This merged vector is then given to the GRU

The shapes of all the vectors at each step have been specified in the comments in the code:

In [17]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz, bidirectional=False, secondlayer=False):
    super(Encoder, self).__init__()
    self.bidirectional=bidirectional
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size,
                                               embedding_dim,
                                               embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                               trainable=False)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform',
                                   dropout=0.3) # TODO I need to make this bidirectional
    if self.bidirectional:
      self.gru = tf.keras.layers.Bidirectional(self.gru)


  def call(self, x, hidden, training=True):
    x = self.embedding(x)
    if self.bidirectional:
      output, forward_state, backward_state = self.gru(x, initial_state = hidden, training= training)
      state = tf.concat([forward_state, backward_state], axis=1)
    else:
      output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    if self.bidirectional:
      return [tf.zeros((self.batch_sz, self.enc_units)) for i in range(2)]
    else:
      return tf.zeros((self.batch_sz, self.enc_units))

In [18]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, bidirectional=False)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 80, 600)
Encoder Hidden state shape: (batch size, units) (64, 600)


In [19]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, attention_type='luong'):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.attention_type = attention_type

    # Embedding Layer
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    # Define the fundamental cell for decoder recurrent structure
    self.gru = tf.keras.layers.GRUCell(self.dec_units,
                                       recurrent_initializer='glorot_uniform')
    # TODO MAKE THIS 2 LAYER???
    
    #Final Dense layer on which softmax will be applied
    self.fc = tf.keras.layers.Dense(vocab_size)

    # Sampler
    self.sampler = tfa.seq2seq.sampler.TrainingSampler()

    # Create attention mechanism with memory = None
    self.attention_mechanism = self.build_attention_mechanism(self.dec_units, 
                                                              None, self.batch_sz*[max_length_inp], self.attention_type)

    # Wrap attention mechanism with the fundamental rnn cell of decoder
    self.rnn_cell = tfa.seq2seq.AttentionWrapper(self.gru, 
                                  self.attention_mechanism, attention_layer_size=self.dec_units)
    
    # Define the decoder with respect to fundamental rnn cell
    self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)

  def build_attention_mechanism(self, dec_units, memory, memory_sequence_length, attention_type='luong'):
    # ------------- #
    # typ: Which sort of attention (Bahdanau, Luong)
    # dec_units: final dimension of attention outputs 
    # memory: encoder hidden states of shape (batch_size, max_length_input, enc_units)
    # memory_sequence_length: 1d array of shape (batch_size) with every element set to max_length_input (for masking purpose)

    if(attention_type=='bahdanau'):
      return tfa.seq2seq.BahdanauAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
    else:
      return tfa.seq2seq.LuongAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)

  def build_initial_state(self, batch_sz, encoder_state, Dtype):
    decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)
    decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
    return decoder_initial_state

  def call(self, x, hidden):
    # enc_output shape == (batch_size, max_length, hidden_size)
    # context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    #x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    #output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    #output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    # x = self.fc(output)

    #return x, state, attention_weights

    outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_sz*[max_length_targ-1])
    return outputs

In [20]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 'luong')

#sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
#                                      sample_hidden, sample_output)

#print('Decoder output shape: (batch_size, vocab size)', sample_decoder_output.shape)

In [21]:
sample_x = tf.random.uniform((BATCH_SIZE, max_length_targ))
decoder.attention_mechanism.setup_memory(sample_output)
initial_state = decoder.build_initial_state(BATCH_SIZE, sample_hidden, tf.float32)


sample_decoder_outputs = decoder(sample_x, initial_state)

print("Decoder Outputs Shape: ", sample_decoder_outputs.rnn_output.shape)


Decoder Outputs Shape:  (64, 19, 32495)


## Define the optimizer and the loss function

In [22]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [23]:
checkpoint_dir = path_to_model + 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [24]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_input = targ[ : , :-1 ] # Ignore <end> token
    real = targ[ : , 1: ]         # ignore <start> token

    # Set the AttentionMechanism object with encoder_outputs
    decoder.attention_mechanism.setup_memory(enc_output)

    # Create AttentionWrapperState as initial_state for decoder
    decoder_initial_state = decoder.build_initial_state(BATCH_SIZE,enc_hidden, tf.float32)
    pred = decoder(dec_input, decoder_initial_state)
    logits = pred.rnn_output
    loss = loss_function(real, logits)



  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return loss

In [26]:
EPOCHS = 30

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 5 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 6.2187
Epoch 1 Batch 100 Loss 3.9117
Epoch 1 Batch 200 Loss 3.5503
Epoch 1 Batch 300 Loss 3.1473
Epoch 1 Batch 400 Loss 3.3725
Epoch 1 Batch 500 Loss 3.2294
Epoch 1 Batch 600 Loss 2.9548
Epoch 1 Batch 700 Loss 3.0385
Epoch 1 Batch 800 Loss 2.7689
Epoch 1 Batch 900 Loss 2.9449
Epoch 1 Batch 1000 Loss 2.9555
Epoch 1 Loss 3.2039
Time taken for 1 epoch 181.8860490322113 sec

Epoch 2 Batch 0 Loss 2.3056
Epoch 2 Batch 100 Loss 2.3245
Epoch 2 Batch 200 Loss 2.7555
Epoch 2 Batch 300 Loss 2.2199
Epoch 2 Batch 400 Loss 2.2965
Epoch 2 Batch 500 Loss 2.6315
Epoch 2 Batch 600 Loss 2.2704
Epoch 2 Batch 700 Loss 2.2921
Epoch 2 Batch 800 Loss 2.5623
Epoch 2 Batch 900 Loss 2.6030
Epoch 2 Batch 1000 Loss 2.4310
Epoch 2 Loss 2.4637
Time taken for 1 epoch 171.99409890174866 sec

Epoch 3 Batch 0 Loss 1.9673
Epoch 3 Batch 100 Loss 2.0469
Epoch 3 Batch 200 Loss 2.0390
Epoch 3 Batch 300 Loss 2.0529
Epoch 3 Batch 400 Loss 1.9670
Epoch 3 Batch 500 Loss 2.1442
Epoch 3 Batch 600 Loss 2.3701
E

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [25]:
def evaluate_sentence(sentence):
  sentence = qg_dataset.preprocess_sentence(sentence)

  inputs = inp_tokenizer.texts_to_sequences([sentence])
  inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                          maxlen=max_length_inp,
                                                          padding='post')
  inputs = tf.convert_to_tensor(inputs)
  inference_batch_size = inputs.shape[0]
  result = ''

  enc_start_state = [tf.zeros((inference_batch_size, units))]
  enc_out, enc_hidden = encoder(inputs, enc_start_state, training=False)

  dec_hidden = enc_hidden

  start_tokens = tf.fill([inference_batch_size], targ_tokenizer.word_index['<start>'])
  end_token = targ_tokenizer.word_index['<end>']

  greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()

  # Instantiate BasicDecoder object
  decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.rnn_cell, sampler=greedy_sampler, output_layer=decoder.fc, maximum_iterations=10)
  # Setup Memory in decoder stack
  decoder.attention_mechanism.setup_memory(enc_out)

  # set decoder_initial_state
  decoder_initial_state = decoder.build_initial_state(inference_batch_size, dec_hidden, tf.float32)


  ### Since the BasicDecoder wraps around Decoder's rnn cell only, you have to ensure that the inputs to BasicDecoder 
  ### decoding step is output of embedding layer. tfa.seq2seq.GreedyEmbeddingSampler() takes care of this. 
  ### You only need to get the weights of embedding layer, which can be done by decoder.embedding.variables[0] and pass this callabble to BasicDecoder's call() function

  decoder_embedding_matrix = decoder.embedding.variables[0]

  outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, end_token= end_token, initial_state=decoder_initial_state)
  return outputs.sample_id.numpy()

def translate(sentence):
  result = evaluate_sentence(sentence)
  print(result)
  result = targ_tokenizer.sequences_to_texts(result)
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

In [26]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  print(attention.shape)
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [27]:
targ_tokenizer.sequences_to_texts([[192, 29, 4,  29,   4,  29,   4,  29,   4,  29]])

['street with ? with ? with ? with ? with']

## Restore the latest checkpoint and test

In [28]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [29]:
translate(['two', 'months', 'later', 'the', 'band', 'got', 'signed', 'to', 'a', 'three', 'album', 'deal', 'with', ',', 'which', 'left', '.'])

[[ 21 196 402  11 738   8 432   4   3]]
Input: ['two', 'months', 'later', 'the', 'band', 'got', 'signed', 'to', 'a', 'three', 'album', 'deal', 'with', ',', 'which', 'left', '.']
Predicted translation: ['which queen album was signed in 2008 ? <end>']


In [30]:
translate(["Golm", "is", "a", "locality", "of", "Potsdam", ",", "the", "capital", "of", "the", "German", "state", "of", "Brandenburg", "."])

[[  6  10   5 315   7 732 677 435   4   3]]
Input: ['Golm', 'is', 'a', 'locality', 'of', 'Potsdam', ',', 'the', 'capital', 'of', 'the', 'German', 'state', 'of', 'Brandenburg', '.']
Predicted translation: ['what is the capital of so district park ? <end>']


In [31]:
translate("the largest of these is the eldon square shop-ping centre , one of the largest city centre shopping com-plexes in the uk .".split(" "))

[[ 14 635  26   5  47 551 212 225   4   3]]
Input: ['the', 'largest', 'of', 'these', 'is', 'the', 'eldon', 'square', 'shop-ping', 'centre', ',', 'one', 'of', 'the', 'largest', 'city', 'centre', 'shopping', 'com-plexes', 'in', 'the', 'uk', '.']
Predicted translation: ['how far does the city run through southampton ? <end>']


In [32]:
def beam_evaluate_sentence(sentence, beam_width=3):
  sentence = qg_dataset.preprocess_sentence(sentence)

  inputs = inp_tokenizer.texts_to_sequences([sentence])
  inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                          maxlen=max_length_inp,
                                                          padding='post')
  inputs = tf.convert_to_tensor(inputs)
  inference_batch_size = inputs.shape[0]
  result = []

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden

  start_tokens = tf.fill([inference_batch_size], targ_tokenizer.word_index['<start>'])
  end_token = targ_tokenizer.word_index['<end>']

  # From official documentation
  # NOTE If you are using the BeamSearchDecoder with a cell wrapped in AttentionWrapper, then you must ensure that:
  # The encoder output has been tiled to beam_width via tfa.seq2seq.tile_batch (NOT tf.tile).
  # The batch_size argument passed to the get_initial_state method of this wrapper is equal to true_batch_size * beam_width.
  # The initial state created with get_initial_state above contains a cell_state value containing properly tiled final state from the encoder.

  enc_out = tfa.seq2seq.tile_batch(enc_out, multiplier=beam_width)
  decoder.attention_mechanism.setup_memory(enc_out)
  print("beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 16, 1024]] :", enc_out.shape)

  # set decoder_inital_state which is an AttentionWrapperState considering beam_width
  hidden_state = tfa.seq2seq.tile_batch(enc_hidden, multiplier=beam_width)
  decoder_initial_state = decoder.rnn_cell.get_initial_state(batch_size=beam_width*inference_batch_size, dtype=tf.float32)
  decoder_initial_state = decoder_initial_state.clone(cell_state=hidden_state)

  # Instantiate BeamSearchDecoder
  decoder_instance = tfa.seq2seq.BeamSearchDecoder(decoder.rnn_cell,beam_width=beam_width, output_layer=decoder.fc, maximum_iterations=10)
  decoder_embedding_matrix = decoder.embedding.variables[0]

  # The BeamSearchDecoder object's call() function takes care of everything.
  outputs, final_state, sequence_lengths = decoder_instance(decoder_embedding_matrix, start_tokens=start_tokens, end_token=end_token, initial_state=decoder_initial_state)
  # outputs is tfa.seq2seq.FinalBeamSearchDecoderOutput object. 
  # The final beam predictions are stored in outputs.predicted_id
  # outputs.beam_search_decoder_output is a tfa.seq2seq.BeamSearchDecoderOutput object which keep tracks of beam_scores and parent_ids while performing a beam decoding step
  # final_state = tfa.seq2seq.BeamSearchDecoderState object.
  # Sequence Length = [inference_batch_size, beam_width] details the maximum length of the beams that are generated


  # outputs.predicted_id.shape = (inference_batch_size, time_step_outputs, beam_width)
  # outputs.beam_search_decoder_output.scores.shape = (inference_batch_size, time_step_outputs, beam_width)
  # Convert the shape of outputs and beam_scores to (inference_batch_size, beam_width, time_step_outputs)
  final_outputs = tf.transpose(outputs.predicted_ids, perm=(0,2,1))
  beam_scores = tf.transpose(outputs.beam_search_decoder_output.scores, perm=(0,2,1))

  return final_outputs.numpy(), beam_scores.numpy()


In [33]:
def beam_translate(sentence):
  result, beam_scores = beam_evaluate_sentence(sentence)
  print(result.shape, beam_scores.shape)
  for beam, score in zip(result, beam_scores):
    print(beam.shape, score.shape)
    output = targ_tokenizer.sequences_to_texts(beam)
    output = [a for a in output]
    beam_score = [a.sum() for a in score]
    print('Input: %s' % (sentence))
    for i in range(len(output)):
      print('{} Predicted translation: {}  {}'.format(i+1, output[i], beam_score[i]))


In [34]:
beam_translate("the largest of these is the eldon square shop-ping centre , one of the largest city centre shopping com-plexes in the uk .".split(" "))

beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 16, 1024]] : (3, 80, 600)
(1, 3, 10) (1, 3, 10)
(3, 10) (3, 10)
Input: ['the', 'largest', 'of', 'these', 'is', 'the', 'eldon', 'square', 'shop-ping', 'centre', ',', 'one', 'of', 'the', 'largest', 'city', 'centre', 'shopping', 'com-plexes', 'in', 'the', 'uk', '.']
1 Predicted translation: how far is the boundaries for new gods ? <end>  -27.01357650756836
2 Predicted translation: how far is the boundaries for the mod ' ?  -38.39814758300781
3 Predicted translation: how far is the boundaries for the mod ' independence  -40.65667724609375


In [35]:
len(targ_tokenizer.word_index)

32494

In [36]:
beam_translate(['Golm', 'is', 'a', 'locality', 'of', 'Potsdam', ',', 'the', 'capital', 'of', 'the', 'German', 'state', 'of', 'Brandenburg', '.'])

beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 16, 1024]] : (3, 80, 600)
(1, 3, 8) (1, 3, 8)
(3, 8) (3, 8)
Input: ['Golm', 'is', 'a', 'locality', 'of', 'Potsdam', ',', 'the', 'capital', 'of', 'the', 'German', 'state', 'of', 'Brandenburg', '.']
1 Predicted translation: what is the capital located ? <end> <end>  -15.25469970703125
2 Predicted translation: what is the capital of bern ? <end>  -25.132383346557617
3 Predicted translation: what is the capital ? <end> <end> <end>  -30.831396102905273
